# Data collecting for model

In [21]:
import requests
import csv
from bs4 import BeautifulSoup

def scrape_product_data(product_id):
    url = f"https://mcdonalds.az/products/{product_id}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve product {product_id}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract product name
    name_tag = soup.find('h1', class_='heading typo-h2')
    name = name_tag.text.strip() if name_tag else "Unknown"
    
    # Extract description
    description_tag = soup.find('p')
    description = description_tag.text.strip() if description_tag else "No description"
    
    # Extract allergens
    allergens = ""
    for p in soup.find_all('p'):
        if "Allergenlər" in p.text:
            allergens = p.text.replace("Allergenlər:", "").strip()
            break
    
    # Extract nutritional values
    nutrition = {}
    for p in soup.find_all('p'):
        if "Zülal" in p.text:
            lines = p.text.split('<br>')
            for line in lines:
                parts = line.split(' - ')
                if len(parts) == 2:
                    key, value = parts[0].strip(), parts[1].strip()
                    nutrition[key] = value
    
    return {
        "ID": product_id,
        "Name": name,
        "Description": description,
        "Allergens": allergens,
        "Protein (g)": nutrition.get("Zülal (qr)", "N/A"),
        "Fat (g)": nutrition.get("Yağlar (qr)", "N/A"),
        "Carbs (g)": nutrition.get("Karbohidrat (qr)", "N/A"),
        "Energy (kcal)": nutrition.get("Enerji dəyəri (kkal)", "N/A")
    }

def save_to_csv(data, filename="mcdonalds_products.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    scraped_data = []
    
    for product_id in range(10, 242):
        product_data = scrape_product_data(product_id)
        if product_data:
            scraped_data.append(product_data)
    
    if scraped_data:
        save_to_csv(scraped_data)
        print("Data saved to mcdonalds_products.csv")
    else:
        print("No data scraped.")

Failed to retrieve product 134
Failed to retrieve product 135
Failed to retrieve product 136
Failed to retrieve product 139
Data saved to mcdonalds_products.csv


In [6]:
import pandas as pd 

## Lots of NaN values how to fix this?

In [4]:
data = pd.read_csv("mcdonalds_products.csv")
data.head()

,ID,Name,Description,Allergens,Protein (g),Fat (g),Carbs (g),Energy (kcal)
0,10,Hamburger,Karamelləşdirilmiş bulkanın arasında 100% mal ...,"Xardal, Qlüten.",NaN,NaN,NaN,NaN
1,11,Dabl Hamburger,Karamelləşdirilmiş bulkanın arasında iki 100% ...,"Xardal, Qlüten.",NaN,NaN,NaN,NaN
2,12,Tripl Hamburger,Yumşaq bulkasının arasında 3 ədəd 100% mal əti...,"Xardal, Qlüten.",NaN,NaN,NaN,NaN
3,13,Çizburger,Karamelləşdirilmiş bulkanın arasında 100% mal ...,"Xardal, Süd, Soya, Yumurta, Qlüten.",NaN,NaN,NaN,NaN
4,14,Dabl Çizburger,Karamelləşdirilmiş bulkanın arasında iki ədəd ...,"Xardal, Süd, Soya, Yumurta, Qlüten.",NaN,NaN,NaN,NaN


# Collecting data from for missing part

In [ ]:
def scrape_product_data(product_id):
    url = f"https://mcdonalds.az/products/{product_id}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve product {product_id}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract product name
    name_tag = soup.find('h1', class_='heading typo-h2')
    name = name_tag.text.strip() if name_tag else "Unknown"
    
    # Extract full nutritional data as a single string
    nutrition_text = "No nutritional info"
    for p in soup.find_all("p"):
        if "Zülal" in p.text:  # This ensures we pick the correct paragraph
            nutrition_text = p.text.strip()
            break  # Stop after finding the first match

    return {
        "ID": product_id,
        "Name": name,
        "Nutritional Info": nutrition_text  # Store full text instead of parsing numbers
    }

def save_to_csv(data, filename="mcdonalds_nutrition1.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    scraped_data = []
    
    for product_id in range(10, 242):  # Only scrape 10 to 20 for testing
        product_data = scrape_product_data(product_id)
        if product_data:
            scraped_data.append(product_data)
    
    if scraped_data:
        save_to_csv(scraped_data)
        print("Data saved to mcdonalds_nutrition1.csv")
    else:
        print("No data scraped.")


Failed to retrieve product 134
Failed to retrieve product 135
Failed to retrieve product 136
Failed to retrieve product 139
Data saved to mcdonalds_nutrition.csv


In [9]:
data2 = pd.read_csv("mcdonalds_nutrition1.csv")
data2.head()

,ID,Name,Nutritional Info
0,10,Hamburger,"Zülal (qr) - 13,2\r\nYağlar (qr) - 9,3\r\nKarb..."
1,11,Dabl Hamburger,Zülal (qr) - 20\r\nYağlar (qr) - 16\r\nKarbohi...
2,12,Tripl Hamburger,Zülal (qr) - 27\r\nYağlar (qr) - 20\r\nKarbohi...
3,13,Çizburger,"Zülal (qr) - 16,1\r\nYağlar (qr) - 13,1\r\nKar..."
4,14,Dabl Çizburger,Zülal (qr) - 27\r\nYağlar (qr) - 22\r\nKarbohi...


In [25]:
data2[("Nutritional Info")][0]

'Zülal (qr) - 13,2\r\nYağlar (qr) - 9,3\r\nKarbohidrat (qr) - 27,7\r\nEnerji dəyəri (qr) - 252'

## Using regular expression to split text and numbers

In [ ]:
import pandas as pd
import re

# Load your CSV file into a DataFrame
df = pd.read_csv('mcdonalds_nutrition1.csv')

# Function to extract nutritional values and convert to float
def extract_nutritional_values(nutrition_text):
    # Define a regular expression to extract key-value pairs
    nutrition_data = re.findall(r'([A-Za-zğöçş]+)\s*\(.*\)\s*-\s*([\d,]+)', nutrition_text)
    
    # Convert the list of tuples into a dictionary, replacing comma with dot for float conversion
    nutrition_dict = {key: float(value.replace(',', '.')) for key, value in nutrition_data}
    
    return nutrition_dict

# Apply the function to the "Nutritional Info" column
df['Nutritional Values'] = df['Nutritional Info'].apply(extract_nutritional_values)

# If you'd like to split each value into a new column:
nutrition_cols = df['Nutritional Values'].apply(pd.Series)

# Now, the DataFrame 'nutrition_cols' contains the extracted numbers as floats
df = pd.concat([df, nutrition_cols], axis=1)

# Optionally, you can save the DataFrame with the new columns
df.to_csv('mcdonalds_nutrition_processed.csv', index=False)

# Show the resulting DataFrame
print(df.head())


   ID             Name                                   Nutritional Info  \
0  10        Hamburger  Zülal (qr) - 13,2\r\nYağlar (qr) - 9,3\r\nKarb...   
1  11   Dabl Hamburger  Zülal (qr) - 20\r\nYağlar (qr) - 16\r\nKarbohi...   
2  12  Tripl Hamburger  Zülal (qr) - 27\r\nYağlar (qr) - 20\r\nKarbohi...   
3  13        Çizburger  Zülal (qr) - 16,1\r\nYağlar (qr) - 13,1\r\nKar...   
4  14   Dabl Çizburger  Zülal (qr) - 27\r\nYağlar (qr) - 22\r\nKarbohi...   

                                  Nutritional Values   lal  Yağlar  \
0  {'lal': 13.2, 'Yağlar': 9.3, 'Karbohidrat': 27...  13.2     9.3   
1  {'lal': 20.0, 'Yağlar': 16.0, 'Karbohidrat': 3...  20.0    16.0   
2  {'lal': 27.0, 'Yağlar': 20.0, 'Karbohidrat': 3...  27.0    20.0   
3  {'lal': 16.1, 'Yağlar': 13.1, 'Karbohidrat': 2...  16.1    13.1   
4  {'lal': 27.0, 'Yağlar': 22.0, 'Karbohidrat': 3...  27.0    22.0   

   Karbohidrat     ri  
0         27.7  252.0  
1         31.0  350.0  
2         31.0  420.0  
3         28.5  300.

## Manually add values to csv file

In [4]:
import pandas as pd
data = pd.read_csv("mcdonalds_products.csv")
data.head()

,ID,Name,Description,Allergens,Protein (g),Fat (g),Carbs (g),Energy (kcal)
0,10,Hamburger,Karamelləşdirilmiş bulkanın arasında 100% mal ...,"Xardal, Qlüten.",13.2,9.3,27.7,252.0
1,11,Dabl Hamburger,Karamelləşdirilmiş bulkanın arasında iki 100% ...,"Xardal, Qlüten.",20.0,16.0,31.0,350.0
2,12,Tripl Hamburger,Yumşaq bulkasının arasında 3 ədəd 100% mal əti...,"Xardal, Qlüten.",27.0,20.0,31.0,420.0
3,13,Çizburger,Karamelləşdirilmiş bulkanın arasında 100% mal ...,"Xardal, Süd, Soya, Yumurta, Qlüten.",16.1,13.1,28.5,300.0
4,14,Dabl Çizburger,Karamelləşdirilmiş bulkanın arasında iki ədəd ...,"Xardal, Süd, Soya, Yumurta, Qlüten.",27.0,22.0,34.0,443.0


### Checking missing values in dataset

In [5]:
data.isnull().any()

ID               False
Name             False
Description       True
Allergens         True
Protein (g)       True
Fat (g)           True
Carbs (g)         True
Energy (kcal)     True
dtype: bool

In [8]:
data.Name[0]

'Hamburger'

In [19]:
# for i in range(100):
#     print(data.Name[i])

null_values = len(data[data["Name"] == "Unknown"])
null_values


0

In [18]:
data = data[data["Name"] != "Unknown"]

In [21]:
data

,ID,Name,Description,Allergens,Protein (g),Fat (g),Carbs (g),Energy (kcal)
0,10,Hamburger,Karamelləşdirilmiş bulkanın arasında 100% mal ...,"Xardal, Qlüten.",13.2,9.3,27.7,252.0
1,11,Dabl Hamburger,Karamelləşdirilmiş bulkanın arasında iki 100% ...,"Xardal, Qlüten.",20.0,16.0,31.0,350.0
2,12,Tripl Hamburger,Yumşaq bulkasının arasında 3 ədəd 100% mal əti...,"Xardal, Qlüten.",27.0,20.0,31.0,420.0
3,13,Çizburger,Karamelləşdirilmiş bulkanın arasında 100% mal ...,"Xardal, Süd, Soya, Yumurta, Qlüten.",16.1,13.1,28.5,300.0
4,14,Dabl Çizburger,Karamelləşdirilmiş bulkanın arasında iki ədəd ...,"Xardal, Süd, Soya, Yumurta, Qlüten.",27.0,22.0,34.0,443.0
...,...,...,...,...,...,...,...,...
223,237,Şampinyon delüks (toyuq),Karamelləşdirilmiş bulkanın arasında qızardılm...,"Xardal ,Yumurta, Süd, Kərəviz, Glüten.",33.0,32.0,64.0,679.0
224,238,Mini Tasty,NaN,"Süd, Yumurta, Qlüten.",31.0,14.0,36.0,473.0
225,239,Dabl Mini Tasty,"Yumşaq bulka, 2 ədəd 100% mal ətindən bifşteks...","Süd, Yumurta, Qlüten.",62.0,28.0,72.0,946.0
226,240,Acılı sous,Dadlı və ətirli sous,NaN,0.0,0.0,11.0,47.0


In [ ]:
data

'Vanil krem ilə donat'

### Divide dataset to Burgers and Other

In [29]:
# Define the full list of burger names
burger_names = [
    "Şaurma Roll", "Mini Tasty", "Dabl Mini Tasty", "Hamburger", "Dabl Hamburger", 
    "Çizburger", "Tripl Hamburger", "Dabl Çizburger", "Tripl Çizburger", "Big Mac™", 
    "Dabl Big Mac™", "Royal Çizburger", "Dabl Royal Çizburger", "Big Tasty™ (Ət)", 
    "Dabl Big Tasty™ (Ət)", "Big Tasty™ (Toyuq)", "Dabl Big Tasty™ (Toyuq)", 
    "McChicken™", "Sezar Roll", "Dabl McChicken™", "Çikenburger", "Dabl Çikenburger", 
    "Filet-O-Fish™", "Dabl Filet-O-Fish™", "Bif Freş", "Çiken Freş"
]

# Filter data for burgers
burger_data = data[data["Name"].isin(burger_names)]

# Filter data for non-burgers (everything else)
other_data = data[~data["Name"].isin(burger_names)]

# Group the burger data by "Name"
grouped_burgers = burger_data.groupby("Name").agg({
    'Description': 'first',  # You can change aggregation as needed
    'Allergens': 'first',
    'Protein (g)': 'mean',
    'Fat (g)': 'mean',
    'Carbs (g)': 'mean',
    'Energy (kcal)': 'mean'
}).reset_index()

# Group the other items by "Name"
grouped_others = other_data.groupby("Name").agg({
    'Description': 'first',
    'Allergens': 'first',
    'Protein (g)': 'mean',
    'Fat (g)': 'mean',
    'Carbs (g)': 'mean',
    'Energy (kcal)': 'mean'
}).reset_index()

# Display the result for burgers and others
print("Grouped Burgers:")
print(grouped_burgers)

print("\nGrouped Others:")
print(grouped_others)


Grouped Burgers:
                       Name  \
0                  Bif Freş   
1                  Big Mac™   
2        Big Tasty™ (Toyuq)   
3           Big Tasty™ (Ət)   
4             Dabl Big Mac™   
5   Dabl Big Tasty™ (Toyuq)   
6      Dabl Big Tasty™ (Ət)   
7        Dabl Filet-O-Fish™   
8            Dabl Hamburger   
9           Dabl McChicken™   
10          Dabl Mini Tasty   
11     Dabl Royal Çizburger   
12         Dabl Çikenburger   
13           Dabl Çizburger   
14            Filet-O-Fish™   
15                Hamburger   
16               McChicken™   
17               Mini Tasty   
18          Royal Çizburger   
19               Sezar Roll   
20          Tripl Hamburger   
21          Tripl Çizburger   
22               Çiken Freş   
23              Çikenburger   
24                Çizburger   
25              Şaurma Roll   

                                          Description  \
0   Karamelləşdirilmiş küncütlü bulkanın arasında ...   
1   Küncütlü bulkanın arasında 

In [35]:
print(grouped_burgers.shape)
grouped_burgers


(26, 7)


,Name,Description,Allergens,Protein (g),Fat (g),Carbs (g),Energy (kcal)
0,Bif Freş,Karamelləşdirilmiş küncütlü bulkanın arasında ...,"Xardal, Süd, Kərəviz, Yumurta, Qlüten, Küncüt ...",NaN,NaN,NaN,NaN
1,Big Mac™,Küncütlü bulkanın arasında iki ədəd 100% mal ə...,"Xardal, Süd, Soya, Yumurta, Qlüten, Küncüt tum...",28.0,28.0,48.0,558.0
2,Big Tasty™ (Toyuq),Həcmli küncütlü bulka arasında toyuqdan olan k...,"Süd, Yumurta, Qlüten, Küncüt tumları.",44.0,48.0,50.0,812.0
3,Big Tasty™ (Ət),"Həcmli küncütlü bulka arasında böyük, 100% mal...","Süd, Yumurta, Qlüten, Küncüt tumları.",44.0,48.0,50.0,812.0
4,Dabl Big Mac™,Küncütlü bulkanın arasında dörd ədəd 100% mal ...,"Xardal, Süd, Soya, Yumurta, Qlüten, Küncüt tum...",38.0,44.0,48.0,740.0
5,Dabl Big Tasty™ (Toyuq),Həcmli küncütlü bulka arasında toyuqdan olan 2...,"Süd, Yumurta, Qlüten, Küncüt tumları.",76.0,30.0,50.0,1114.0
6,Dabl Big Tasty™ (Ət),"Həcmli küncütlü bulka arasında 2 ədəd böyük, 1...","Süd, Yumurta, Qlüten, Küncüt tumları.",76.0,30.0,50.0,1114.0
7,Dabl Filet-O-Fish™,İsti yumşaq bulkanın arasında yaxşıca qızardıl...,"Xardal, Süd, Soya, Yumurta, Balıq, Qlüten.",26.0,21.0,45.0,478.0
8,Dabl Hamburger,Karamelləşdirilmiş bulkanın arasında iki 100% ...,"Xardal, Qlüten.",20.0,16.0,31.0,350.0
9,Dabl McChicken™,Karamelləşdirilmiş bulkanın arasında suxarıya ...,"Xardal, Kərəviz, Yumurta, Qlüten, Küncüt tumları.",32.5,40.3,54.0,712.0


In [36]:
print(grouped_others.shape)
grouped_others

(187, 7)


,Name,Description,Allergens,Protein (g),Fat (g),Carbs (g),Energy (kcal)
0,1000 Ada sousu,Zəngin dadlı iştahaçan sous.,"Süd, Balıq, Qlüten.",NaN,NaN,NaN,NaN
1,Acılı sous,Dadlı və ətirli sous,None,0.0,0.0,11.0,47.0
2,Alma piroqu,"Şəkər xəmiri əsaslı un məmulatı, almalı içlik ...","Süd, Soya, Qlüten, Yumurta.",4.0,14.0,45.0,329.0
3,Amerika sayağı çizkeyk,Şəkər xəmiri üzərində yumşaq qaymaqlı pendir.,"Araxis, Fındıq, Süd, Soya, Yumurta.",8.0,26.0,29.0,394.0
4,Amerikano\n ...,Rainforest Alliance Certified™ ticarət nişanlı...,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
182,Şokoladlı Frappe,Südlü və xoş şokolad dadlı soyuq içki.,"Fındıq, Süd, Soya, Qlüten, Yumurta.",13.0,4.0,67.0,355.0
183,Şokoladlı Tort,"Avstriya tortu ""Zaxer""in resepti əsasında hazı...","Araxis, Süd, Soya, Qlüten, Küncüt tumları.",3.0,14.0,52.0,351.0
184,Şokoladlı krem ilə donat,Şokoladlı içliklə zərif donat.,"Süd, Qlüten.",7.0,30.0,46.0,490.0
185,Şokoladlı peçenye,"İncə şəkərli peçenye, qara, tünd və qarışıq şo...","Süd, Qlüten,Yumurta.",5.0,18.0,58.0,438.0


In [38]:
grouped_burgers.columns

Index(['Name', 'Description', 'Allergens', 'Protein (g)', 'Fat (g)',
       'Carbs (g)', 'Energy (kcal)'],
      dtype='object')